### Installering og brug af Ollama lokalt med python

- Open windows powershell -> type "winget install --id Ollama.Ollama"
- Once complete, open anaconda powershell (or restart windows powershell) and type "ollama pull mistral"
- (Optional): 
    - Create new environment (still in anaconda powershell): type "conda create -n ollama_env python"
    -   Once complete, type "conda activate ollama_env"
- Open your editor of choice and make sure your new environment is active (if you are reading this in a jupyter notebook, make sure to switch to your ollama environment, if you have made one)
- Try to run the cells below

In [2]:
!pip install ollama

In [25]:
import ollama
import pandas as pd


In [4]:
code_prompt = "Write a Python function that checks if a number is prime."
response = ollama.generate(model='mistral', prompt=code_prompt)
print(response['response'])

 Here's a simple Python function to check if a number is prime:

```python
def is_prime(n):
    if n <= 1:
        return False
    elif n <= 3:
        return True
    elif n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True
```

This function works by first checking if the number is less than or equal to 1, in which case it's not prime. Then it checks for small factors like 2 and 3. After that, it skips every other number starting from 5 because every even number greater than 2 can be written as the sum of two primes (e.g., 6 = 3 + 3).

The function then starts checking for divisors by incrementing `i` in steps of 6 since all numbers greater than 5 that are not of the form 4*k+1 or 4*k+3 have a prime factor less than or equal to √n and can be found using this pattern (e.g., 7 is the next prime after 5, 11 is the next prime after 7, and so on).


In [8]:
print("Streaming response:")
for chunk in ollama.generate('mistral', 'Why is the sky yellow?', stream=True):
    print(chunk['response'], end='', flush=True)
print()

Streaming response:
 The sky appears yellow, orange, or red during sunrise and sunset due to a phenomenon called Rayleigh scattering. During these times, sunlight has to pass through more of Earth's atmosphere, and the shorter wavelength blue light is scattered out in all directions while the longer wavelength red, orange, and yellow light are able to travel further towards the observer.

However, if you're asking about a specific instance where the sky appears yellow (not during sunrise or sunset), it could be due to various factors like pollution, dust storms, or even certain weather conditions such as fog or haze. These particles in the air scatter sunlight and can give the sky a yellowish tint.


In [6]:
from ollama import chat

# Define a system prompt
system_prompt = "You speaks and sounds like a pirate with short sentences."
# Chat with a system prompt
response = chat('mistral', 
                messages=[
                    {'role': 'system', 'content': system_prompt},
                    {'role': 'user', 'content': 'Tell me about your boat.'}
                ])
print(response.message.content)

 Ahoy matey! Me ship be a fine vessel, yarr! A sturdy galleon with sails that catch the wind like a hungry shark, and timbers as strong as Old Captain Morgan's rum! Below decks, there's a treasure trove of supplies to keep me crew fed and watered. The cannon be ready for any scallywags who cross our path. And don't forget about the Jolly Roger flying high, striking fear into the hearts of them landlubbers we pass! Arr!


In [11]:
# Example: Summarize a paragraph of text
text = """
OpenAI has introduced a new tool called Ollama that lets users run large language models on local machines.
This approach emphasizes privacy and control, as data does not leave the user's environment.
Developers can leverage various open-source models through a simple interface, improving efficiency and reducing costs.
"""
prompt = f"Summarize the following text in one sentence:\n\"\"\"\n{text}\n\"\"\""
result = ollama.generate(model='mistral', prompt=prompt)
print("Summary:", result['response'])

Summary:  OpenAI has released Ollama, a tool that allows users to run large language models on their local machines for enhanced privacy, control, efficiency, and cost reduction by utilizing multiple open-source models via a user-friendly interface.


In [ ]:
response = ollama.generate(model='mistral', prompt='Generate a list of 100 string elements with random sentences containing the name of a color somewhere')
sentences = pd.Series(response['response'].split('.'))
sentences = sentences[sentences.str.len()>10].iloc[1:]
df = sentences.to_frame(name='comment').reset_index(drop=True)
df

,comment
1,The sky turned a vibrant shade of purple at s...
3,Red roses adorned the table for the special d...
5,A brilliant orange sun rose above the horizon
7,She wore a bright yellow dress to stand out i...
9,"The ocean water was a deep blue, perfect for ..."
...,...
191,The artist used a palette of earthy tones to ...
193,A brilliant stroke of blue paint added life t...
195,He decided to dye his hair an electric shade ...
197,The fireworks display painted the night sky w...


In [30]:
# df = pd.DataFrame({
#     "comment": [
#         "Why is the bike pink.",
#         "The color of the phone box is red.",
#         "I really like green cars.",
#         "Her dress was bright yellow.",
#         "The wall is painted white."
#     ]
# })
# create a new empty column for the LLM result
df["color"] = None
# creat a standard global prompt
SYSTEM_PROMPT = (
    "Extract the color mentioned in the text. "
    "Reply with ONLY the color word in lowercase (e.g., blue, red). "
    "If no color is mentioned, reply with 'none'."
)

# loop though comments and prompt the model
for i, comment in df["comment"].items():
    # prompt = f"{SYSTEM_PROMPT}\n\nText: {comment}"
    
    resp = ollama.chat(
        model="mistral",
        messages=[{'role': 'system', 'content': SYSTEM_PROMPT},
                    {'role': 'user', 'content': comment}],
        stream=False,
        options={
          # "temperature": 0.1, # introduce randomness
          "seed": 123, # set seed to be able to replicate results
          },
    )
    color = resp["message"]["content"].strip().lower()
    df.at[i, "color"] = color # insert te answer ine the new col
print(df)


                                               comment              color
1     The sky turned a vibrant shade of purple at s...             purple
3     Red roses adorned the table for the special d...                red
5        A brilliant orange sun rose above the horizon             orange
7     She wore a bright yellow dress to stand out i...             yellow
9     The ocean water was a deep blue, perfect for ...               blue
..                                                 ...                ...
191   The artist used a palette of earthy tones to ...              brown
193   A brilliant stroke of blue paint added life t...               blue
195   He decided to dye his hair an electric shade ...             purple
197   The fireworks display painted the night sky w...  red, white, green
199   She loved the way the morning sunlight filled...             golden

[100 rows x 2 columns]
